In [ ]:
# Import necessary modules
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import sys

here = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.abspath(os.path.join(here, '../../'))
sys.path.insert(0, project_root)

from core.mpc.mpc import MPC
from core.mpc.mpc_params import MPCParams
from core.mpc.mpc_bounds import ControlInputBounds

from core.model.model_carrying_capacities import ModelCarryingCapacities
from core.model.model_disturbances import ModelDisturbances
from core.model.model_growth_rates import ModelGrowthRates
from core.model.model_initial_conditions import ModelInitialConditions
from core.model.model_params import ModelParams
from core.model.model_typical_disturbances import ModelTypicalDisturbances
from core.model.model_sensitivities import ModelSensitivities

from core.ga.ga_member import Member
from core.ga.ga_params import GeneticAlgorithmParams

from core.plotting.plotting import *

## Model Parameters

In [ ]:
# Set ModelParams (time stepping)
model_params = ModelParams(
    dt               = 1,  # hours/step
    simulation_hours = 2900, # hours
    closed_form      = False,
    verbose          = True
)

In [ ]:
# Set ModelCarryingCapacities
carrying_capacities = ModelCarryingCapacities(
    kh = 3.0,  # m
    kA = 0.65, # m2
    kN = 20,   # number of leaves
    kc = 1000, # number of spikelets
    kP = 0.25  # kg
)

In [ ]:
# Set ModelGrowthRates
growth_rates = ModelGrowthRates(
    ah = 0.01,   # 1/hr
    aA = 0.0105, # 1/hr
    aN = 0.011,  # 1/hr
    ac = 0.01,   # 1/hr
    aP = 0.005   # 1/hr
)

In [ ]:
# Set ModelSensitivities
sensitivities = ModelSensitivities(
    sigma_W = 30,  # hrs
    sigma_F = 300, # hrs
    sigma_T = 30,  # hrs
    sigma_R = 30   # hrs
)

In [ ]:
# Set ModelInitialConditions
initial_conditions = ModelInitialConditions(
    h0=carrying_capacities.kh/model_params.simulation_hours, # m/hr
    A0=carrying_capacities.kA/model_params.simulation_hours, # m2/hr
    N0=carrying_capacities.kN/model_params.simulation_hours, # number/hr
    c0=carrying_capacities.kc/model_params.simulation_hours, # number/hr
    P0=carrying_capacities.kP/model_params.simulation_hours  # kg/hr
)

## MPC params and bounds

In [ ]:
mpc_params = MPCParams(
    hourly_horizon=10,
    solver          = "ipopt",
    solver_options  = {
        "tol": 1e-4,          # looser tolerance than default 1e-8
        "acceptable_iter": 100, # stop earlier if acceptable
        "max_iter": 500,      # limit iterations so it doesn't grind forever
        "print_level": 5,     # moderate verbosity
        "mu_strategy": "adaptive",
        "linear_solver": "mumps",
    },
    reoptimization_interval=5
)
bounds = ControlInputBounds()

## Input Disturbances

In [ ]:
# Hourly precipitation, radiation, and temperature from CSV
hourly_input_disturbances = pd.read_csv(
    '../../io/inputs/hourly_prcp_rad_temp_iowa.csv'
)

In [ ]:
# Set ModelDisturbances
input_disturbances = ModelDisturbances(
    precipitation = 0.5*hourly_input_disturbances['Hourly Precipitation (in)'].to_numpy(), # imagine it's a drought year
    radiation     = hourly_input_disturbances['Hourly Radiation (W/m2)'].to_numpy(),
    temperature   = hourly_input_disturbances['Temperature (C)'].to_numpy()
)

## Plot disturbances

In [ ]:
plot_hourly_inputs(
    input_disturbances
)

In [ ]:
# Convert inputs from hourly to simulation time steps
from core.model.model_helpers import get_sim_inputs_from_hourly
simulation_hours = len(input_disturbances.precipitation)
temperature   = get_sim_inputs_from_hourly(input_disturbances.temperature,   model_params.dt, simulation_hours, mode='split')
radiation     = get_sim_inputs_from_hourly(input_disturbances.radiation,     model_params.dt, simulation_hours, mode='split')
precipitation = get_sim_inputs_from_hourly(input_disturbances.precipitation, model_params.dt, simulation_hours, mode='split')

## Optimal/Typical Values

In [ ]:
default_typical_disturbances = ModelTypicalDisturbances()

typical_disturbances = ModelTypicalDisturbances(
    typical_water       = default_typical_disturbances.typical_water * model_params.dt,
    typical_fertilizer  = default_typical_disturbances.typical_fertilizer * model_params.dt,
    typical_temperature = default_typical_disturbances.typical_temperature * model_params.dt,
    typical_radiation   = default_typical_disturbances.typical_radiation * model_params.dt
)

## Simulation of Plant Growth over Season

In [ ]:
# Construct an instance of the Member class and get the cost
mpc = MPC(
    carrying_capacities  = carrying_capacities,
    disturbances         = input_disturbances,
    growth_rates         = growth_rates,
    initial_conditions   = initial_conditions,
    model_params         = model_params,
    typical_disturbances = typical_disturbances,
    sensitivities        = sensitivities,
    mpc_params           = mpc_params,
    bounds               = bounds
)

In [ ]:
# Run the MPC
mpc_result = mpc.run()

# Save the result(
with open('mpc_nonlinear_result.pkl', 'wb') as f:
    pickle.dump(mpc_result, f)

# Read-in the results for plotting

In [ ]:
# Open the pickled MPC result
with open('mpc_nonlinear_result.pkl', 'rb') as f:
    mpc_result = pickle.load(f)

In [ ]:
h = mpc_result["h"]
A = mpc_result["A"]
N = mpc_result["N"]
c = mpc_result["c"]
P = mpc_result["P"]

irrigation = mpc_result["irrigation"]
fertilizer = mpc_result["fertilizer"]

delayed_water = mpc_result["logs"]["delayed_water"]
delayed_fertilizer = mpc_result["logs"]["delayed_fertilizer"]
delayed_temperature = mpc_result["logs"]["delayed_temperature"]
delayed_radiation = mpc_result["logs"]["delayed_radiation"]

cumulative_water = mpc_result["logs"]["cumulative_water"]
cumulative_fertilizer = mpc_result["logs"]["cumulative_fertilizer"]
cumulative_temperature = mpc_result["logs"]["cumulative_temperature"]
cumulative_radiation = mpc_result["logs"]["cumulative_radiation"]

cumulative_divergence_water = mpc_result["logs"]["cumulative_divergence_water"]
cumulative_divergence_fertilizer = mpc_result["logs"]["cumulative_divergence_fertilizer"]
cumulative_divergence_temperature = mpc_result["logs"]["cumulative_divergence_temperature"]
cumulative_divergence_radiation = mpc_result["logs"]["cumulative_divergence_radiation"]

nuWs = mpc_result["logs"]["nuW"]
nuFs = mpc_result["logs"]["nuF"]
nuTs = mpc_result["logs"]["nuR"]
nuRs = mpc_result["logs"]["nuT"]

## Plot crop growth results

In [ ]:
plot_crop_growth_results(
    hs = [h],
    As = [A],
    Ns = [N],
    cs = [c],
    Ps = [P]
)

In [ ]:
ga_params = GeneticAlgorithmParams()

profit = ga_params.weight_fruit_biomass * P[-1]
expenses = (ga_params.weight_irrigation * np.sum(irrigation)
            + ga_params.weight_fertilizer * np.sum(fertilizer))
revenue = profit - expenses

print(f'Height: {h[-1]}')
print(f'Fruit biomass: {P[-1]}')

print(f'Irrigation used (in): {np.sum(irrigation):.2f}, irrigation expense: ${ga_params.weight_irrigation * np.sum(irrigation):.2f}')
print(f'Fertilizer used (lbs): {np.sum(fertilizer):.2f}, fertilizer expense: ${ga_params.weight_fertilizer * np.sum(fertilizer):.2f}')

print(f'Profit: ${profit:.2f}')
print(f'Expenses: ${expenses:.2f}')

print(f'\n')

print(f'Revenue: ${revenue:.2f}')

## Plot input and disturbance evolution with delays taken into account

In [ ]:
plot_applied_vs_absorbed(
        irrigation,
        precipitation[0:len(irrigation)],
        delayed_water,
        fertilizer,
        delayed_fertilizer,
        temperature[0:len(irrigation)],
        delayed_temperature,
        radiation[0:len(irrigation)],
        delayed_radiation
)

In [ ]:
# 1) Hourly totals
total_irrigation_hourly = irrigation.sum()
total_fertilizer_hourly = fertilizer.sum()

print("Total irrigation from hourly array:", total_irrigation_hourly)
print("Total fertilizer from hourly array:", total_fertilizer_hourly)

# 2) Cumulative totals used in the second plot
total_water_cum = cumulative_water[-1] - cumulative_water[0]
total_fert_cum  = cumulative_fertilizer[-1]  - cumulative_fertilizer[0]

print("Total water from cumulative curve:", total_water_cum)
print("Total fert from cumulative curve:", total_fert_cum)

# 3) Absorbed/delayed signals logged:
total_absorbed_water = delayed_water.sum()
total_absorbed_fert  = delayed_fertilizer.sum()

print("Total absorbed water:", total_absorbed_water)
print("Total absorbed fert:", total_absorbed_fert)


## Plot cumulative value evolution

In [ ]:
plot_cumulative_values(
    cumulative_water,
    cumulative_fertilizer,
    cumulative_temperature,
    cumulative_radiation,
    typical_disturbances
)

## Plot the deltas between actual and expected cumulative value

In [ ]:
plot_actual_vs_expected_cumulatives(
    cumulative_divergence_water,
    cumulative_divergence_fertilizer,
    cumulative_divergence_temperature,
    cumulative_divergence_radiation
)

## Plot nutrient factor evolution

In [ ]:
plot_nutrient_factor_evolution(
    [nuWs],
    [nuFs],
    [nuTs],
    [nuRs]
)

## Run same simulation with Member cost function from GA

In [ ]:
model_params = ModelParams(
    dt               = 1.0,  # hours/step
    simulation_hours = 2900, # hours
    closed_form      = True,
    verbose          = True
)

ga_params = GeneticAlgorithmParams()
weight_fruit_biomass = ga_params.weight_fruit_biomass
weight_irrigation    = ga_params.weight_irrigation
weight_fertilizer    = ga_params.weight_fertilizer

default_typical_disturbances = ModelTypicalDisturbances()
typical_disturbances = ModelTypicalDisturbances(
    typical_water       = default_typical_disturbances.typical_water * model_params.dt,
    typical_fertilizer  = default_typical_disturbances.typical_fertilizer * model_params.dt,
    typical_temperature = default_typical_disturbances.typical_temperature * model_params.dt,
    typical_radiation   = default_typical_disturbances.typical_radiation * model_params.dt
)

ga_params = GeneticAlgorithmParams()

In [ ]:
appended_irrigation = np.append(irrigation, (model_params.total_time_steps - len(irrigation)) * [0])

appended_irrigation = get_sim_inputs_from_hourly(appended_irrigation, model_params.dt, model_params.simulation_hours, mode='split')

appended_fertilizer = np.append(fertilizer, (model_params.total_time_steps - len(fertilizer)) * [0])
appended_fertilizer = get_sim_inputs_from_hourly(appended_fertilizer, model_params.dt, model_params.simulation_hours, mode='split')

In [ ]:
member = Member(
    ga_params            = ga_params,
    carrying_capacities  = carrying_capacities,
    disturbances         = input_disturbances,
    growth_rates         = growth_rates,
    initial_conditions   = initial_conditions,
    model_params         = model_params,
    typical_disturbances = typical_disturbances,
    sensitivities        = sensitivities,
    values               = np.array([1, 0, 1, 0]) # dummy values
)
member.get_cost(irrigation=appended_irrigation, fertilizer=appended_fertilizer)

In [ ]:
# Read results from the output csv file
output = pd.read_csv('output_get_cost.csv')

h = output['h'].to_numpy()
A = output['A'].to_numpy()
N = output['N'].to_numpy()
c = output['c'].to_numpy()
P = output['P'].to_numpy()

delayed_water       = output['delayed_water'].to_numpy()
delayed_fertilizer  = output['delayed_fertilizer'].to_numpy()
delayed_temperature = output['delayed_temperature'].to_numpy()
delayed_radiation   = output['delayed_radiation'].to_numpy()

cumulative_water       = output['cumulative_water'].to_numpy()
cumulative_fertilizer  = output['cumulative_fertilizer'].to_numpy()
cumulative_temperature = output['cumulative_temperature'].to_numpy()
cumulative_radiation   = output['cumulative_radiation'].to_numpy()

cumulative_divergence_water       = output['cumulative_divergence_water'].to_numpy()
cumulative_divergence_fertilizer  = output['cumulative_divergence_fertilizer'].to_numpy()
cumulative_divergence_temperature = output['cumulative_divergence_temperature'].to_numpy()
cumulative_divergence_radiation   = output['cumulative_divergence_radiation'].to_numpy()

nuWs = output['nuW'].to_numpy()
nuFs = output['nuF'].to_numpy()
nuTs = output['nuT'].to_numpy()
nuRs = output['nuR'].to_numpy()

ah_hat = output['ah_hat'].to_numpy()
aA_hat = output['aA_hat'].to_numpy()
aN_hat = output['aN_hat'].to_numpy()
ac_hat = output['ac_hat'].to_numpy()
aP_hat = output['aP_hat'].to_numpy()

kh_hat = output['kh_hat'].to_numpy()
kA_hat = output['kA_hat'].to_numpy()
kN_hat = output['kN_hat'].to_numpy()
kc_hat = output['kc_hat'].to_numpy()
kP_hat = output['kP_hat'].to_numpy()

In [ ]:
plot_crop_growth_results(
    hs = [h],
    As = [A],
    Ns = [N],
    cs = [c],
    Ps = [P]
)

In [ ]:
profit = weight_fruit_biomass * P[-1]
expenses = (weight_irrigation * np.sum(irrigation)
            + weight_fertilizer * np.sum(fertilizer))
revenue = profit - expenses

print(f'weight_fruit_biomass: {weight_fruit_biomass}')
print(f'weight_irrigation: {weight_irrigation}')
print(f'weight_fertilizer: {weight_fertilizer}')
print(f'Fruit biomass: {P[-1]}')

print(f'\n')

print(f'Profit: ${profit:.2f}')
print(f'Irrigation used (in): {np.sum(irrigation):.2f}, irrigation expense: ${weight_irrigation * np.sum(irrigation):.2f}')
print(f'Fertilizer used (lbs): {np.sum(fertilizer):.2f}, fertilizer expense: ${weight_fertilizer * np.sum(fertilizer):.2f}')
print(f'Expenses: ${expenses:.2f}')
print(f'Revenue: ${revenue:.2f}')

In [ ]:
irrigation    = get_sim_inputs_from_hourly(appended_irrigation,              model_params.dt, model_params.simulation_hours, mode='split')
fertilizer    = get_sim_inputs_from_hourly(appended_fertilizer,              model_params.dt, model_params.simulation_hours, mode='split')
temperature   = get_sim_inputs_from_hourly(input_disturbances.temperature,   model_params.dt, model_params.simulation_hours, mode='split')
radiation     = get_sim_inputs_from_hourly(input_disturbances.radiation,     model_params.dt, model_params.simulation_hours, mode='split')
precipitation = get_sim_inputs_from_hourly(input_disturbances.precipitation, model_params.dt, model_params.simulation_hours, mode='split')

plot_applied_vs_absorbed(
        irrigation,
        precipitation[0:len(irrigation)],
        delayed_water,
        fertilizer,
        delayed_fertilizer,
        temperature,
        delayed_temperature,
        radiation,
        delayed_radiation
)

In [ ]:
plot_cumulative_values(
    cumulative_water,
    cumulative_fertilizer,
    cumulative_temperature,
    cumulative_radiation,
    typical_disturbances
)

In [ ]:
plot_actual_vs_expected_cumulatives(
    cumulative_divergence_water,
    cumulative_divergence_fertilizer,
    cumulative_divergence_temperature,
    cumulative_divergence_radiation
)

In [ ]:
plot_nutrient_factor_evolution(
    [nuWs],
    [nuFs],
    [nuTs],
    [nuRs]
)

## Plot growth rate evolution

In [ ]:
plot_growth_rate_evolution(
        growth_rates,
        [ah_hat],
        [aA_hat],
        [aN_hat],
        [ac_hat],
        [aP_hat],
)

## Plot carrying capacity evolution

In [ ]:
plot_carrying_capacity_evolution(
        carrying_capacities,
        [kh_hat],
        [kA_hat],
        [kN_hat],
        [kc_hat],
        [kP_hat]
)